### import libraries

In [43]:
! pip install netCDF4

In [2]:
import netCDF4 # python API to work with netcdf (.nc) files
import os
import datetime
from osgeo import gdal, ogr, osr
import numpy as np # library to work with matrixes and computations in general
import matplotlib.pyplot as plt # plotting library
from auxiliary_classes import convert_time,convert_time_reverse,kelvin_to_celsius,kelvin_to_celsius_vector,Grid,Image,subImage
import json
import geojson, gdal, subprocess

### auxiliary functions

In [25]:
def print_geojson(tname, tvalue, fname, longitude, latitude, startdoc, position,endloop): #for printing to geojson - start,end,attributes
    fname = fname +".geojson"
    pmode="a"
    if startdoc==1:
        with open(fname, mode="w", encoding='utf-8') as f1: #start of geojson
            tstring = "{\n\"type\": \"FeatureCollection\",\n\"features\": ["
            print(tstring, file=f1)
            f1.close()
    else:
        if position==0: #for printing to geojson - geometry, longitude, latitude
            tstring = "\"type\": \"Feature\",\n\"geometry\": {\n\"type\": \"Point\",\n\"coordinates\": [" + str(longitude) + ","+ str(latitude) + "]\n},\n\"properties\": {"
            fname = fname 
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                print(tstring, file=f1)
                f1.close()
        elif position==1:  #start of point attributes
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                print("{", file=f1)
                f1.close()  
        elif position==2: #print attribute (not last)
             with open(fname, mode=pmode, encoding='utf-8') as f1:
                ttext = "\"" + str(tname) + "\": \"" +str(tvalue) + "\","
                print(ttext, file=f1) 
                f1.close() 
        elif position==3: #print last attribute
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                ttext = "\"" + str(tname) + "\": \"" +str(tvalue) + "\""
                print(ttext, file=f1) 
                f1.close()        
        elif position==4: #end of point attributes
            with open(fname, mode=pmode, encoding='utf-8') as f1:  
                if endloop==0:
                    print("}\n},", file=f1)
                    f1.close()
                else:  #end of geojson
                    print("}\n}\n]\n}", file=f1)
                    f1.close()  

In [26]:
def trend(inputlist, nametrend, namediff, fname): 
    listlong = len(inputlist)
    if listlong <= 1:
        trendcoef = 0
        timediff = 0
        
    else:
        x = np.arange(0,len(inputlist))
        y = inputlist
        z = np.polyfit(x,y,1)
        trendcoef=z[0]
        timediff=int(trendcoef*(listlong-1))
    print_geojson(nametrend, trendcoef, fname, 0, 0, 0, 2, 0)
    print_geojson(namediff, timediff, fname, 0, 0, 0, 3, 0)
    

In [27]:
def trend2(inputlist, nametrend, namediff, endyear, startyear, fname):
    listlong = endyear-startyear+1
    numberweeks = len(inputlist[0])
    for j in range(0, numberweeks,1):
        tempweek = j +1
        if listlong <= 1:
            trendcoef = 0
            timediff = 0
            
        else:
            x = np.arange(0,listlong)
            y = []
            for i in range(0, listlong, 1): 
                y.append( inputlist[i][j])
            z = np.polyfit(x,y,1)
            trendcoef=z[0]
            timediff=int(trendcoef*(listlong-1))
        nametrend2 = nametrend + str(tempweek)
        namediff2 = namediff + str(tempweek)
        print_geojson(nametrend2, trendcoef, fname, 0, 0, 0, 2, 0)
        if j == (numberweeks-1):
             print_geojson(namediff2, timediff, fname, 0, 0, 0, 3, 0)
        else:
            print_geojson(namediff2, timediff, fname, 0, 0, 0, 2, 0)
       
    
    

In [28]:
def avg2Dlist(inputlist,startyear,endyear): #average for 2D list ->1D list # inputs: inputlist = 2D list, output: avglist = 1D list with avg values
    numberyear = endyear-startyear+1
    listlen = len(inputlist[0])
    templist = []  
    avglist = []  
    for i in range(0, listlen,1):
        for j in range(0, numberyear,1):
            templist.append(inputlist[j][i])
        tempvalue=sum(templist)/len(templist)
        avglist.append(tempvalue) 
        templist = [] 
    return avglist
            

In [29]:
def acumulatelist(inputlist): #average for 2D list ->1D list # inputs: inputlist = 2D list, output: avglist = 1D list with avg values
    listlen = len(inputlist)
    for i in range (0,listlen-1,1):
        inputlist[i+1] += inputlist[i]
    return inputlist

In [30]:
def printlistasweekgeojson(inputlist,name,fname,endtemp): # from list of week values print geojson
    listlen = len(inputlist)
    for i in range(0, listlen,1):
        tempvalue=inputlist[i]
        tvarname = name + str(i+1)
        if endtemp==1 and i == (listlen-1):
            print_geojson(tvarname, tempvalue, fname, 0, 0, 0, 3, 0)
        else:
            print_geojson(tvarname, tempvalue, fname, 0, 0, 0, 2, 0)
        
    

###  GDU, HSU, LGP: function for one place

In [31]:
from datetime import date, timedelta
def finddu(latitude,longitude,year,endyear,im,enddate, startdate, fnameHSU,tparam, fnameoptLGP,allweekHSUlist,yearHSUlist,allweekGDUlist,yearGDUlist,allweekoptLGPlist,yearoptLGPlist,allweekabsLGPlist, yearabsLGPlist,fnameGDU, absgrowmax, absgrowmin, optgrowmax, optgrowmin,fnameabsLGP, fnameHSUaccu,fnameGDUaccu,fnameoptLGPaccu,fnameabsLGPaccu,fnameannualHSU,fnameannualGDU, fnameannualoptLGP,fnameannualabsLGP ):
    
    
    sdate = startdate   # start date for searching last frost date
    edate = enddate   # end date for searching last frost date
    delta = edate - sdate       # as timedelta
    sevendays=0 # for determination of new week (1-7)
    currentweek=1 # for determination of weeks
    weekHSU=0
    weekHSUlist = []
    weekGDU=0
    weekGDUlist = []
    weekabsLGP=0
    weekabsLGPlist = []
    weekoptLGP=0
    weekoptLGPlist = []
        
    starthourday = 0
    endhourday = 23
    
    yearHSUsum = 0
    yearGDUsum = 0
    yearabsLGPsum = 0
    yearoptLGPsum = 0
    
    for i in range(delta.days+1):
        daylong = sdate + timedelta(days=i)
        sdaylong = str(daylong)
        tday = int(sdaylong[8:10])
        tmonth = int(sdaylong[5:7])
        tyear = int(sdaylong[0:4])
        daytempsum = 0 # start value
        sevendays+=1
        daymax = 0
        dayavg = 0
        dayHSU = 0
        dayGDU = 0
        dayabsLGP = 0
        dayoptLGP = 0
                
        for hour in range(starthourday, endhourday+1, 1): # for specific hours (all day,only sunrise hours,..)
            time=convert_time_reverse(datetime.datetime(tyear, tmonth, tday, hour, 0)) 
            slice_dictionary={'lon':[longitude,],'lat':[latitude],'time':[int(time)]}
            currenttemp=int(kelvin_to_celsius_vector(im.slice(tparam,slice_dictionary)))
            daytempsum += currenttemp
            if currenttemp > daymax:
                daymax = currenttemp
                
        dayavg = (daytempsum/24)
        
        #HSU
        if daymax > optgrowmax:
            dayHSU = (daymax-absgrowmax)
            
        #GDU + LGP growing
        if dayavg < absgrowmax:
            if dayavg > absgrowmin:
                dayGDU = (dayavg-absgrowmin)
                yearabsLGPsum +=1
                dayabsLGP +=1
        #LGP optimum growing
        if dayavg < optgrowmax:
            if dayavg > optgrowmin:
                yearoptLGPsum += 1
                dayoptLGP += 1
            
        yearHSUsum += dayHSU
        yearGDUsum += dayGDU
                    
               
                
        if daylong == edate: # save week date for last date in season
            weekHSU+=dayHSU
            weekHSUlist.append(weekHSU)
            weekGDU+=dayGDU
            weekGDUlist.append(weekGDU)
            weekoptLGP+=dayoptLGP
            weekoptLGPlist.append(weekoptLGP)
            weekabsLGP+=dayabsLGP
            weekabsLGPlist.append(weekabsLGP)
            
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekHSU, fnameHSU, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekGDU, fnameGDU, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearHSUsum, fnameHSUaccu, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearGDUsum, fnameGDUaccu, 0, 0, 0, 2, 0)
                        
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekoptLGP, fnameoptLGP, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekabsLGP, fnameabsLGP, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearoptLGPsum, fnameoptLGPaccu, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearabsLGPsum, fnameabsLGPaccu, 0, 0, 0, 2, 0)
            
            
        elif sevendays<=7:  # new week?   
            weekHSU+=dayHSU
            weekGDU+=dayGDU
            weekoptLGP+=dayoptLGP
            weekabsLGP+=dayabsLGP
            
        else:
            weekHSUlist.append(weekHSU)
            weekGDUlist.append(weekGDU)
            weekoptLGPlist.append(weekoptLGP)
            weekabsLGPlist.append(weekabsLGP)
            
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekHSU, fnameHSU, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekGDU, fnameGDU, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearHSUsum, fnameHSUaccu, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearGDUsum, fnameGDUaccu, 0, 0, 0, 2, 0)
                        
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekoptLGP, fnameoptLGP, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekabsLGP, fnameabsLGP, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearoptLGPsum, fnameoptLGPaccu, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearabsLGPsum, fnameabsLGPaccu, 0, 0, 0, 2, 0)
            
            weekHSU=dayHSU
            weekGDU=dayGDU
            weekoptLGP=dayoptLGP
            weekabsLGP=dayabsLGP
                
            sevendays=0
            currentweek+=1
        
                    
          
    allweekHSUlist.append(weekHSUlist)
    yearHSUlist.append(yearHSUsum)
    allweekGDUlist.append(weekGDUlist)
    yearGDUlist.append(yearGDUsum)
    allweekoptLGPlist.append(weekoptLGPlist)
    yearoptLGPlist.append(yearoptLGPsum)
    allweekabsLGPlist.append(weekabsLGPlist)
    yearabsLGPlist.append(yearabsLGPsum)
    
    tvarname = "HSU" + str(year) 
    print_geojson(tvarname, yearHSUsum, fnameannualHSU, 0, 0, 0, 2, 0)
    tvarname = "GDU" + str(year) 
    print_geojson(tvarname, yearGDUsum, fnameannualGDU, 0, 0, 0, 2, 0)
    tvarname = "LGP" + str(year) 
    print_geojson(tvarname, yearoptLGPsum, fnameannualoptLGP, 0, 0, 0, 2, 0)
    tvarname = "LGP" + str(year) 
    print_geojson(tvarname, yearabsLGPsum, fnameannualabsLGP, 0, 0, 0, 2, 0)
    
   
       
    
    
    
    
                  

### Find GDU, HSU, LGP: function for selected years

In [32]:
def duyearly(latorder,lonorder,startyear,endyear,endloop,datafolder,fnameHSU,enddatem, startdatem,enddated, startdated,tparam, fnameoptLGP,fnameGDU, absgrowmax, absgrowmin, optgrowmax, optgrowmin,fnameabsLGP, fnameHSUaccu,fnameGDUaccu,fnameoptLGPaccu,fnameabsLGPaccu,fnameannualHSU,fnameannualGDU, fnameannualoptLGP,fnameannualabsLGP ):
    print_geojson("", "", fnameHSU, 0, 0, 0, 1,0)
    print_geojson("", "", fnameGDU, 0, 0, 0, 1,0)
    print_geojson("", "", fnameHSUaccu, 0, 0, 0, 1,0)
    print_geojson("", "", fnameGDUaccu, 0, 0, 0, 1,0)
    print_geojson("", "", fnameannualHSU, 0, 0, 0, 1,0)
    print_geojson("", "", fnameannualGDU, 0, 0, 0, 1,0)
    print_geojson("", "", fnameoptLGP, 0, 0, 0, 1,0)
    print_geojson("", "", fnameabsLGP, 0, 0, 0, 1,0)
    print_geojson("", "", fnameoptLGPaccu, 0, 0, 0, 1,0)
    print_geojson("", "", fnameabsLGPaccu, 0, 0, 0, 1,0)
    print_geojson("", "", fnameannualoptLGP, 0, 0, 0, 1,0)
    print_geojson("", "", fnameannualabsLGP, 0, 0, 0, 1,0)
    
   
    
    
    
    allweekHSUlist=[] # 2D list for all weeks many years
    yearHSUlist=[] # 2D list for all weeks many years
    allweekGDUlist=[] # 2D list for all weeks many years
    yearGDUlist=[] # 2D list for all weeks many years
    allweekoptLGPlist=[] # 2D list for all weeks many years
    yearoptLGPlist=[] # 2D list for all weeks many years
    allweekabsLGPlist=[] # 2D list for all weeks many years
    yearabsLGPlist=[] # 2D list for all weeks many years
     
       
    
    for year in range(startyear, endyear+1, 1):
        source = datafolder + '/' + str(year) + '.nc' 
        im=Image(netCDF4.Dataset(source,'r'))   
        longlist = im.get_data().variables['lon'][:]
        latlist= im.get_data().variables['lat'][:]
        longitude = longlist [lonorder]   
        latitude = latlist[latorder]
        if year == startyear:
            print_geojson("", "", fnameHSU, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameHSUaccu, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameannualHSU, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameGDU, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameGDUaccu, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameannualGDU, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameoptLGP, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameoptLGPaccu, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameannualoptLGP, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameabsLGP, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameabsLGPaccu, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameannualabsLGP, longitude, latitude, 0, 0,0)
            
            
            
        
        enddate=date(year, enddatem, enddated) 
        startdate=date(year, startdatem, startdated) 
                  
        
        
        
                 
            
        finddu(latitude,longitude,year,endyear,im,enddate, startdate, fnameHSU,tparam, fnameoptLGP,allweekHSUlist,yearHSUlist,allweekGDUlist,yearGDUlist,allweekoptLGPlist,yearoptLGPlist,allweekabsLGPlist, yearabsLGPlist,fnameGDU, absgrowmax, absgrowmin, optgrowmax, optgrowmin,fnameabsLGP, fnameHSUaccu,fnameGDUaccu,fnameoptLGPaccu,fnameabsLGPaccu,fnameannualHSU,fnameannualGDU, fnameannualoptLGP,fnameannualabsLGP )
    
    
     
    
    
    avgweekHSUlist = avg2Dlist(allweekHSUlist,startyear,endyear)
    printlistasweekgeojson(avgweekHSUlist,"HSUW",fnameHSU,0)
    avgweekacuHSUlist = acumulatelist(avgweekHSUlist)
    printlistasweekgeojson(avgweekacuHSUlist,"HSUW",fnameHSUaccu,1)
    avgHSUyear=sum(yearHSUlist)/len(yearHSUlist)
    print_geojson("AvgHSU", avgHSUyear, fnameannualHSU, 0, 0, 0, 2, 0)
    nametrend = "AnTrCo"
    namediff = "Andiff"
    trend(yearHSUlist, nametrend, namediff,fnameannualHSU)
    nametrend = "TrCo"
    namediff = "Diff"
    trend2(allweekHSUlist, nametrend, namediff, endyear, startyear, fnameHSU)
    
    avgweekGDUlist = avg2Dlist(allweekGDUlist,startyear,endyear)
    printlistasweekgeojson(avgweekGDUlist,"GDUW",fnameGDU,0)
    avgweekacuGDUlist = acumulatelist(avgweekGDUlist)
    printlistasweekgeojson(avgweekacuGDUlist,"GDUW",fnameGDUaccu,1)
    avgGDUyear=sum(yearGDUlist)/len(yearGDUlist)
    print_geojson("AvgGDU", avgGDUyear, fnameannualGDU, 0, 0, 0, 2, 0)
    nametrend = "AnTrCo"
    namediff = "Andiff"
    trend(yearGDUlist, nametrend, namediff,fnameannualGDU)
    nametrend = "TrCo"
    namediff = "Diff"
    trend2(allweekGDUlist, nametrend, namediff, endyear, startyear, fnameGDU)
    
    avgweekoptLGPlist = avg2Dlist(allweekoptLGPlist,startyear,endyear)
    printlistasweekgeojson(avgweekoptLGPlist,"LOGPW",fnameoptLGP,0)
    avgweekacuoptLGPlist = acumulatelist(avgweekoptLGPlist)
    printlistasweekgeojson(avgweekacuoptLGPlist,"LOGPW",fnameoptLGPaccu,1)
    avgoptLGPyear=sum(yearoptLGPlist)/len(yearoptLGPlist)
    print_geojson("AvgOLGP", avgoptLGPyear, fnameannualoptLGP, 0, 0, 0, 2, 0)
    nametrend = "AnTrCo"
    namediff = "Andiff"
    trend(yearoptLGPlist, nametrend, namediff,fnameannualoptLGP)
    nametrend = "TrCo"
    namediff = "Diff"
    trend2(allweekoptLGPlist, nametrend, namediff, endyear, startyear, fnameoptLGP)
    
    avgweekabsLGPlist = avg2Dlist(allweekabsLGPlist,startyear,endyear)
    printlistasweekgeojson(avgweekabsLGPlist,"LGPW",fnameabsLGP,0)
    avgweekacuabsLGPlist = acumulatelist(avgweekabsLGPlist)
    printlistasweekgeojson(avgweekacuabsLGPlist,"LGPW",fnameabsLGPaccu,1)
    avgabsLGPyear=sum(yearabsLGPlist)/len(yearabsLGPlist)
    print_geojson("AvgLGP", avgabsLGPyear, fnameannualabsLGP, 0, 0, 0, 2, 0)
    nametrend = "AnTrCo"
    namediff = "Andiff"
    trend(yearabsLGPlist, nametrend, namediff,fnameannualabsLGP)
    nametrend = "TrCo"
    namediff = "Diff"
    trend2(allweekabsLGPlist, nametrend, namediff, endyear, startyear, fnameabsLGP)
    
    
   
    print_geojson("", "", fnameHSU, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameGDU, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameHSUaccu, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameGDUaccu, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameannualHSU, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameannualGDU, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameoptLGP, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameabsLGP, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameoptLGPaccu, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameabsLGPaccu, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameannualoptLGP, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameannualabsLGP, 0, 0, 0, 4,endloop)
          

### Find GDU, HSU, LGP: function for selected latitudes, longitudes

In [33]:
def duplaces(startlat, startlon, endlat, endlon, startyear,endyear,exportfolder,datafolder,fnameHSU1,enddatem, startdatem,enddated, startdated, alllatlonfile, tparam, fnameoptLGP1,fnameGDU1,absgrowmax, absgrowmin, optgrowmax, optgrowmin,fnameabsLGP1, fnameHSUaccu1,fnameGDUaccu1,fnameoptLGPaccu1,fnameabsLGPaccu1,fnameannualHSU1,fnameannualGDU1, fnameannualoptLGP1,fnameannualabsLGP1):
        fnameHSU= exportfolder + "/" +fnameHSU1
        fnameHSUaccu= exportfolder + "/" +fnameHSUaccu1
        fnameannualHSU= exportfolder + "/" +fnameannualHSU1
        fnameGDU= exportfolder + "/" +fnameGDU1
        fnameGDUaccu= exportfolder + "/" +fnameGDUaccu1
        fnameannualGDU= exportfolder + "/" +fnameannualGDU1
        fnameabsLGP= exportfolder + "/" +fnameabsLGP1
        fnameabsLGPaccu= exportfolder + "/" +fnameabsLGPaccu1
        fnameannualabsLGP= exportfolder + "/" +fnameannualabsLGP1
        fnameoptLGP= exportfolder + "/" +fnameoptLGP1
        fnameoptLGPaccu= exportfolder + "/" +fnameoptLGPaccu1
        fnameannualoptLGP= exportfolder + "/" +fnameannualoptLGP1
        
       
        
         #start in geojson files:
        print_geojson("", "", fnameHSU, 0, 0, 1, 0,0)
        print_geojson("", "", fnameGDU, 0, 0, 1, 0,0)
        print_geojson("", "", fnameHSUaccu, 0, 0, 1, 0,0)
        print_geojson("", "", fnameGDUaccu, 0, 0, 1, 0,0)
        print_geojson("", "", fnameannualHSU, 0, 0, 1, 0,0)
        print_geojson("", "", fnameannualGDU, 0, 0, 1, 0,0)
        print_geojson("", "", fnameoptLGP, 0, 0, 1, 0,0)
        print_geojson("", "", fnameabsLGP, 0, 0, 1, 0,0)
        print_geojson("", "", fnameoptLGPaccu, 0, 0, 1, 0,0)
        print_geojson("", "", fnameabsLGPaccu, 0, 0, 1, 0,0)
        print_geojson("", "", fnameannualoptLGP, 0, 0, 1, 0,0)
        print_geojson("", "", fnameannualabsLGP, 0, 0, 1, 0,0)
                
        endloop=0
        
        if alllatlonfile==1:  # if it is calculated for all latitudes and longitudes in input file
            source = datafolder + '/' + str(startyear) + '.nc' 
            im=Image(netCDF4.Dataset(source,'r')) 
            arraylon = im.get_data().variables['lon'][0::]
            arraylat = im.get_data().variables['lat'][0::]
            startlat=0
            startlon=0
            endlon= len(arraylon)-1
            endlat= len(arraylat)-1
        
        for latorder in range(startlat, endlat+1, 1):
            for lonorder in range(startlon, endlon+1, 1):
                if latorder==endlat and lonorder==endlon:
                    endloop=1
                duyearly(latorder,lonorder,startyear,endyear,endloop,datafolder,fnameHSU,enddatem, startdatem,enddated, startdated,tparam, fnameoptLGP,fnameGDU,absgrowmax, absgrowmin, optgrowmax, optgrowmin,fnameabsLGP, fnameHSUaccu,fnameGDUaccu,fnameoptLGPaccu,fnameabsLGPaccu,fnameannualHSU,fnameannualGDU, fnameannualoptLGP,fnameannualabsLGP )
       
       
        

## <font color=red>Find GDU, HSU, LGP: input parameters and launch</font> 

In [34]:
#Time definition:
startyear=2017 #start year (integer) 
endyear=2018 #end year (integer)  
enddatem = 12 # start date (month) each year
enddated = 31  # start date (day) each year
startdatem = 1 # end date (month) each year
startdated = 1 # end date (day) each year


#Temperatures of growing (degrees of Celsia):
absgrowmax = 30
absgrowmin = 10
optgrowmax = 33
optgrowmin = 18

#Files/Folders name:
datafolder = "data" #folder with data files (named by year) for each year #string
fnameHSU ="HSU" #name of created files with HSU
fnameGDU ="GDU" #name of created files with GDU
fnameoptLGP ="LOGP" #name of created files with number of optimal growing days
fnameabsLGP ="LGP" #name of created files with number of growing days
fnameHSUaccu ="HSU_accumulated" #name of created files with HSU
fnameGDUaccu ="GDU_accumulated" #name of created files with GDU
fnameoptLGPaccu ="LOGP_accumulated" #name of created files with number of optimal growing days
fnameabsLGPaccu ="LGP_accumulated" #name of created files with number of growing days
fnameannualHSU ="annual_HSU" #name of created files with HSU
fnameannualGDU ="annual_GDU" #name of created files with GDU
fnameannualoptLGP ="annual_LOGP" #name of created files with number of optimal growing days
fnameannualabsLGP ="annual_LGP" #name of created files with number of growing dayss

exportfolder = "export" #for all files (if each file its folder -> insert name of folder to name of file) #export folder must be created #string

#Area definition:
alllatlonfile=0 #calculate all latitudes and longitudes in input file (1=yes, 0=no)
# if alllatlonfile!=0 then:
startlat=0 # start number of list of latitudes from used netCDF4 file 
startlon=0 # start number of list of longitudes from used netCDF4 file 
endlat=1 # end number of list of latitudes from used netCDF4 file 
endlon=1 # end number of list of longitudes from used netCDF4 file 

#Temperature data parameter:
tparam = 'tas'


duplaces(startlat, startlon, endlat, endlon, startyear,endyear,exportfolder,datafolder,fnameHSU,enddatem, startdatem,enddated, startdated,alllatlonfile,tparam, fnameoptLGP, fnameGDU, absgrowmax, absgrowmin, optgrowmax, optgrowmin,fnameabsLGP, fnameHSUaccu,fnameGDUaccu,fnameoptLGPaccu,fnameabsLGPaccu,fnameannualHSU,fnameannualGDU, fnameannualoptLGP,fnameannualabsLGP)






## From geojson to shp

In [35]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/HSU.shp', 'export/HSU.geojson']
subprocess.Popen(args)

In [36]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/GDU.shp', 'export/GDU.geojson']
subprocess.Popen(args)

In [37]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/LOGP.shp', 'export/LOGP.geojson']
subprocess.Popen(args)

In [38]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/LGP.shp', 'export/LGP.geojson']
subprocess.Popen(args)

In [39]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/HSU_accumulated.shp', 'export/HSU_accumulated.geojson']
subprocess.Popen(args)

In [40]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/GDU_accumulated.shp', 'export/GDU_accumulated.geojson']
subprocess.Popen(args)

In [41]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/LOGP_accumulated.shp', 'export/LOGP_accumulated.geojson']
subprocess.Popen(args)

In [3]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/vyberC3opt.shp', 'export/vyberC3opt.geojson']
subprocess.Popen(args)

In [43]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/annual_HSU.shp', 'export/annual_HSU.geojson']
subprocess.Popen(args)

In [44]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/annual_GDU.shp', 'export/annual_GDU.geojson']
subprocess.Popen(args)

In [45]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/annual_LOGP.shp', 'export/annual_LOGP.geojson']
subprocess.Popen(args)

In [46]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/annual_LGP.shp', 'export/annual_LGP.geojson']
subprocess.Popen(args)